<a href="https://colab.research.google.com/github/Myavuzars/NLP-with-Tensorflow/blob/main/Sentiment_Analysis_Amazon_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
import pandas as pd

In [2]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P

--2021-03-27 17:07:26--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 216.58.192.110, 2607:f8b0:4026:803::200e
Connecting to drive.google.com (drive.google.com)|216.58.192.110|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vqv3okul0tid083qk6gd1k2jfcmq8qib/1616864775000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P [following]
--2021-03-27 17:07:26--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vqv3okul0tid083qk6gd1k2jfcmq8qib/1616864775000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 74.125.137.132, 2607:f8b0:4023:c03::84
Connecting to doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)|74.125.137.132|:4

In [3]:
dataset = pd.read_csv('/tmp/sentiment.csv')
dataset.head(10)

,Unnamed: 0,text,sentiment
0,0,So there is no way for me to plug it in here i...,0
1,1,Good case Excellent value.,1
2,2,Great for the jawbone.,1
3,3,Tied to charger for conversations lasting more...,0
4,4,The mic is great.,1
5,5,I have to jiggle the plug to get it to line up...,0
6,6,If you have several dozen or several hundred c...,0
7,7,If you are Razr owner...you must have this!,1
8,8,Needless to say I wasted my money.,0
9,9,What a waste of money and time!.,0


In [4]:
sentences = dataset['text'].tolist()
labels= dataset['sentiment'].tolist()

In [5]:
print(len(labels))

1992


In [6]:
#Data Preparation
size= int(len(labels)*0.8)

training_sentences =sentences[:size]
training_labels = labels[:size]

test_sentences = sentences[size:]
test_labels = labels[size:]

#Make labels np array
test_labels_array = np.array(test_labels)
train_labels_array =np.array(training_labels)


In [8]:
#Hyperparameters
vocab_size= 1000
embedding_dim = 16
max_length = 100
trunc_type ='post'
padding_type= 'post'
oov_tok = "<OOV>"



In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded= pad_sequences(sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)

testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded= pad_sequences(testing_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)


In [10]:
#Review a Sequence
reverse_word_index =dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'.') for i in text])

print(decode_review(training_padded[1]))
print(training_sentences[1])

good case excellent value . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Good case Excellent value.


In [16]:
#Building a model

model= tf.keras.Sequential([
                            tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                            tf.keras.layers.Dropout(0.4),
                            tf.keras.layers.Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'),
                            tf.keras.layers.Dropout(0.2),
                            tf.keras.layers.Conv1D(filters=16, kernel_size=2, padding='same', activation='relu'),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(6,activation='relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')

])

model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 16)           16000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 16)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 100, 32)           1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 16)           1040      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                

In [17]:
num_epohs =20

model.fit(training_padded,train_labels_array, epochs=num_epohs, validation_data=(testing_padded, test_labels_array) )

Epoch 1/20
50/50 [==============================] - 1s 15ms/step - loss: 0.6936 - accuracy: 0.5093 - val_loss: 0.6963 - val_accuracy: 0.4110
Epoch 2/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6910 - accuracy: 0.5379 - val_loss: 0.6951 - val_accuracy: 0.4110
Epoch 3/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6865 - accuracy: 0.5287 - val_loss: 0.6952 - val_accuracy: 0.4110
Epoch 4/20
50/50 [==============================] - 1s 11ms/step - loss: 0.6420 - accuracy: 0.5566 - val_loss: 0.6594 - val_accuracy: 0.6266
Epoch 5/20
50/50 [==============================] - 1s 11ms/step - loss: 0.5271 - accuracy: 0.7902 - val_loss: 0.6622 - val_accuracy: 0.6767
Epoch 6/20
50/50 [==============================] - 1s 11ms/step - loss: 0.4705 - accuracy: 0.8317 - val_loss: 0.7141 - val_accuracy: 0.6892
Epoch 7/20
50/50 [==============================] - 1s 12ms/step - loss: 0.4363 - accuracy: 0.8749 - val_loss: 0.6208 - val_accuracy: 0.7168
Epoch 8/20
50

In [18]:
#Prdeciting Sentiment analysis
fake_reviews= ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']

In [19]:
# Create the sequences
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(fake_reviews)
fakes_padded = pad_sequences(sample_sequences, padding=padding_type, maxlen=max_length)           

classes = model.predict(fakes_padded)

# The closer the class is to 1, the more positive the review is deemed to be
for x in range(len(fake_reviews)):
  print(fake_reviews[x])
  if classes[x] < 0.5:
    print(classes[x],":  NEGATIVE REVIEW" )
    print('\n')
  else:
    print(classes[x],":  POSITIVE REVIEW" )
    print('\n')

# Try adding reviews of your own
# Add some negative words (such as "not") to the good reviews and see what happens
# For example:
# they gave us free chocolate cake and did not charge us

I love this phone
[0.9999989] :  POSITIVE REVIEW


I hate spaghetti
[0.3012939] :  NEGATIVE REVIEW


Everything was cold
[0.3012939] :  NEGATIVE REVIEW


Everything was hot exactly as I wanted
[0.7748083] :  POSITIVE REVIEW


Everything was green
[0.3012939] :  NEGATIVE REVIEW


the host seated us immediately
[0.9997903] :  POSITIVE REVIEW


they gave us free chocolate cake
[0.99998295] :  POSITIVE REVIEW


not sure about the wilted flowers on the table
[0.3012939] :  NEGATIVE REVIEW


only works when I stand on tippy toes
[0.99988896] :  POSITIVE REVIEW


does not work when I stand on my head
[0.3012939] :  NEGATIVE REVIEW


